<a href="https://colab.research.google.com/github/GrigoryBartosh/hse07_nlp/blob/master/4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
! pip install transformers

In [0]:
import os
import re
import pandas as pd

import numpy as np
from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import torch.nn.functional as F

from transformers import BertTokenizer, BertModel

from tqdm.auto import tqdm, trange
import matplotlib.pyplot as plt

import logging
logging.basicConfig(level=logging.CRITICAL)

PATH_DATASET = os.path.join('data', 'train_qa.csv')

PATH_DATASET_TEST = os.path.join('data', 'test.txt')
PATH_RESULTS = os.path.join('data', 'results.txt')

MAX_TEXT_LEN = 512

EPOCHS_1 = 40
EPOCHS_2 = 5
BATCH_SIZE = 32
LEARNING_RATE_1 = 0.0001
LEARNING_RATE_2 = 0.0001
W_L2_NORM = 0.0

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [0]:
dataset = pd.read_csv(PATH_DATASET)
dataset_texts = dataset['paragraph']
dataset_questions = dataset['question']
dataset_answers = dataset['answer']

In [0]:
tokenizer = BertTokenizer.from_pretrained(
    'bert-base-multilingual-cased',
    do_lower_case=False
)

In [0]:
def prepare_sample(text, question, answer):
    answer = answer.lower()
    while (answer[0] == '.'):
        answer = answer[1:]
    while (answer[-1] in ['.', '?']):
        answer = answer[:-1]
        
    if answer not in text.lower():
        return [], []
    
    first = text.lower().find(answer)
    last = first + len(answer)
    
    text_1 = text[:first].strip()
    text_2 = text[first:last].strip()
    text_3 = text[last:].strip()
    text_tokens = tokenizer.tokenize(text_1)
    first = len(text_tokens)
    text_tokens += tokenizer.tokenize(text_2)
    last = len(text_tokens) - 1
    text_tokens += tokenizer.tokenize(text_3)
    
    question_tokens = tokenizer.tokenize(question)
    
    length = MAX_TEXT_LEN - len(question_tokens) - 3
    if len(text_tokens) > length:
        part_length = length // 3
        stride = 3 * part_length
        nrow = np.ceil(len(text_tokens) / part_length) - 2
        indexes = part_length * np.arange(nrow)[:, None] + np.arange(stride)
        indexes = indexes.astype(np.int32)

        max_index = indexes.max()
        diff = max_index + 1 - len(text_tokens)
        text_tokens += diff * [tokenizer.pad_token]

        text_tokens = np.array(text_tokens)[indexes].tolist()
        
        tokens = []
        labels = []
        for i, ts in enumerate(text_tokens):
            while ts[-1] == tokenizer.pad_token:
                ts = ts[:-1]
                
            tokens += [ts]
                
            lfirst = first - i * part_length
            llast = last - i * part_length
            
            labels += [(((lfirst if lfirst >= 0 and lfirst < len(ts) else 0) + 2 + len(question_tokens),
                         lfirst >= 0 and lfirst < len(ts)),
                        ((llast if llast >= 0 and llast < len(ts) else 0) + 2 + len(question_tokens),
                         llast >= 0 and llast < len(ts)))]
    else:
        tokens = [text_tokens]
        labels = [((first, 1), (last, 1))]
        
    for i in range(len(tokens)):
        tokens[i] = [tokenizer.cls_token] + \
                    question_tokens + \
                    [tokenizer.sep_token] + \
                    tokens[i] + \
                    [tokenizer.sep_token]

    return tokens, labels

In [0]:
dataset_tokens, dataset_labels = [], []

for text, question, answer in tqdm(zip(dataset_texts, dataset_questions, dataset_answers)):
    tokens, labels = prepare_sample(text, question, answer)
    dataset_tokens += tokens
    dataset_labels += labels 

x_train, x_val, y_train, y_val = train_test_split(dataset_tokens, dataset_labels, test_size=0.1)
train_data = list(zip(x_train, y_train))
val_data = list(zip(x_val, y_val))

In [0]:
def text_collate_fn(texts):
    max_len = max([len(text) for text in texts])
    masks = [[1] * len(text) + [0] * (max_len - len(text)) for text in texts]
    texts = [text + [tokenizer.pad_token] * (max_len - len(text)) for text in texts]
    texts = [tokenizer.convert_tokens_to_ids(text) for text in texts]
    texts = torch.LongTensor(texts)
    masks = torch.LongTensor(masks)

    return texts, masks

def collate_fn(data):
    texts, labels = zip(*data)

    texts, masks = text_collate_fn(texts)
    
    labels_first, labels_last = zip(*labels)
    labels_first_pos, labels_first_valid = zip(*labels_first)
    labels_last_pos, labels_last_valid = zip(*labels_last)
    
    labels_first_pos = torch.LongTensor(labels_first_pos)
    labels_first_mask = torch.LongTensor(labels_first_valid)
    labels_last_pos = torch.LongTensor(labels_last_pos)
    labels_last_mask = torch.LongTensor(labels_last_valid)
    
    return texts, masks, labels_first_pos, labels_first_mask, labels_last_pos, labels_last_mask

train_data_loader = data.DataLoader(
    dataset=train_data,
    batch_size=BATCH_SIZE,
    shuffle=True,
    collate_fn=collate_fn
)
val_data_loader = data.DataLoader(
    dataset=val_data,
    batch_size=BATCH_SIZE,
    shuffle=False,
    collate_fn=collate_fn
)

In [0]:
class TextClassifier(nn.Module):
    def __init__(self):
        super(TextClassifier, self).__init__()
        
        self.bert = BertModel.from_pretrained('bert-base-multilingual-cased')

        for param in self.bert.parameters():
            param.requires_grad = False

        layers = [
            nn.Linear(768, 128),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            nn.Linear(128, 32),
            nn.ReLU(),
            nn.Dropout(p=0.3),
            nn.Linear(32, 16),
            nn.ReLU(),
            nn.Dropout(p=0.2),
            nn.Linear(16, 2)
        ]
        self.layers = nn.Sequential(*layers)

    def forward(self, text, mask):
        x = self.bert(text, attention_mask=mask)[0]
        x = self.layers(x)
        return x

In [0]:
class MaskedLoss(nn.Module):
    EPS  = 1e-8

    def __init__(self):
        super(MaskedLoss, self).__init__()

        self.sigmoid = nn.Sigmoid()

    def masked_softmax(self, vec, mask, dim=1):
        masked_vec = vec * mask.float()
        max_vec = torch.max(masked_vec, dim=dim, keepdim=True)[0]
        exps = torch.exp(masked_vec - max_vec)
        masked_exps = exps * mask.float()
        masked_sums = masked_exps.sum(dim, keepdim=True)
        zeros = (masked_sums == 0)
        masked_sums += zeros.float()
        return masked_exps / masked_sums
        
    def forward(self, output, output_mask, target, target_mask):
        output_s = self.sigmoid(output)

        log_0 = -torch.log(1 - output_s + MaskedLoss.EPS)
        log_0 = log_0 * output_mask

        output_t = torch.gather(output_s, 1, target[:, None]).squeeze()
        log_0_t = -torch.log(1 - output_t + MaskedLoss.EPS)
        log_1_t = -torch.log(output_t + MaskedLoss.EPS)
        log_0_t = log_0_t * target_mask
        log_1_t = log_1_t * target_mask
        
        sm = self.masked_softmax(output, output_mask)
        sm = torch.gather(sm, 1, target[:, None]).squeeze()
        sm = sm * target_mask
        
        loss = (log_0.sum() - log_0_t.sum()) / (output_mask.sum() - target_mask.sum()) + \
               log_1_t.sum() / target_mask.sum() + \
               sm.sum() / target_mask.sum()
        
        return loss

In [0]:
def train(model, criterion, optimizer, epochs):
    losses_train, losses_val = [], []
    for _ in trange(epochs):
        losses = []
        model.train()
        for texts, masks, lfp, lfm, llp, llm in train_data_loader:
            texts = texts.to(device)
            masks = masks.to(device)
            lfp = lfp.to(device)
            lfm = lfm.to(device)
            llp = llp.to(device)
            llm = llm.to(device)
            
            optimizer.zero_grad()

            ps = model(texts, masks)
            loss = criterion(ps[:, :, 0], masks, lfp, lfm) + \
                   criterion(ps[:, :, 1], masks, llp, llm)

            loss.backward()
            optimizer.step()

            losses.append(loss.item())

        losses_train.append(np.array(losses).mean())

        losses = []
        model.eval()
        with torch.no_grad():
            for texts, masks, lfp, lfm, llp, llm in val_data_loader:
                texts = texts.to(device)
                masks = masks.to(device)
                lfp = lfp.to(device)
                lfm = lfm.to(device)
                llp = llp.to(device)
                llm = llm.to(device)
                
                ps = model(texts, masks)
                loss = criterion(ps[:, :, 0], masks, lfp, lfm) + \
                       criterion(ps[:, :, 1], masks, llp, llm)

                losses.append(loss.item())

        losses_val.append(np.array(losses).mean())

    plt.plot(range(epochs), losses_train, label="train")
    plt.plot(range(epochs), losses_val, label="val")
    plt.xlabel('epoch num')
    plt.ylabel('loss')
    plt.legend()
    plt.show()

In [0]:
model = TextClassifier()
model.to(device)

criterion = MaskedLoss()

optimizer = optim.Adam(
    filter(lambda p: p.requires_grad, model.parameters()),
    LEARNING_RATE_1,
    weight_decay=W_L2_NORM
)

#train(model, criterion, optimizer, EPOCHS_1)

In [0]:
train(model, criterion, optimizer, 3)

In [0]:
for param in model.parameters():
    param.requires_grad = True

optimizer = optim.Adam(
    filter(lambda p: p.requires_grad, model.parameters()),
    LEARNING_RATE_2,
    weight_decay=W_L2_NORM
)

train(model, criterion, optimizer, EPOCHS_2)

In [0]:
with open(PATH_DATASET_TEST, 'r') as file:
    dataset_test = file.readlines()[1:]
    
res[]
model.eval()
with torch.no_grad():
    for sample in dataset_test:
        _, question_id, paragraph, question = sample.split('\t')

        tokens = [tokenizer.cls_token] + \
                tokenizer.tokenize(question) + \
                [tokenizer.sep_token] + \
                tokenizer.tokenize(paragraph) + \
                [tokenizer.sep_token]

        if (len(tokens) > MAX_TEXT_LEN):
            res += [question_id + u'я ваще хз']
            continue

        texts, masks = text_collate_fn([tokens])
        texts = texts.to(device)
        masks = masks.to(device)

        ps = model(texts, masks)[0]
        ps = ps.cpu().numpy()
        first = ps[:, 0].argmax()
        last = ps[:, 1].argmax()

        res += [question_id + ' '.join(tokens[first, last])]

res = '\n'.join(res)
with open(PATH_RESULTS, 'w') as file:
    file.write(res)